## Segmenting and Clustering Neighborhoods in Toronto

Week 3 assignment for *pkali*, part #3

In [11]:
import pandas as pd
import numpy as np

In [12]:
from bs4 import BeautifulSoup
import requests
base = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(base).text, 'lxml')
tabs = soup.find("table",{"class":"wikitable sortable"})
df = pd.read_html(str(tabs))[0] #read only the first tab
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True,inplace=True)

In [13]:
geospatial = pd.read_csv('http://cocl.us/Geospatial_data')
df = df.merge(geospatial, left_on='Postal Code', right_on='Postal Code')